### imports

In [1]:
import sys

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard.plugins.hparams import api as hp
from PIL import Image
from collections.abc import Mapping

import sklearn

sys.path.append('../')
import models


### Generators for loading and augmenting images

In [2]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                      shear_range=0.1,
#                                      zoom_range=0.1,
#                                      rotation_range=5,
#                                      width_shift_range=0.1,
#                                      height_shift_range=0.1,
#                                      fill_mode='nearest')
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/train',
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../data/test',
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')
                                            

Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


### Setup for experiment

In [3]:
HP_NUM_FREEZED = hp.HParam('num_freezed', hp.Discrete([3, 5]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'RMSprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_FREEZED, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

2022-03-30 11:22:19.444603: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-30 11:22:19.445559: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 11:22:19.447541: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Training function

In [4]:
def train_test_model(hparams):
    model = models.XrayModelEfficient(hparams[HP_NUM_FREEZED])
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=hparams[HP_OPTIMIZER],
                  metrics='accuracy')

    model.fit(train_generator,
              steps_per_epoch=100,
              epochs=1)
    
    _, accuracy = model.evaluate(test_generator)
    return accuracy

### Run experiments

In [5]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [6]:
session_num = 0

for num_freezed in HP_NUM_FREEZED.domain.values:
  for optimizer in HP_OPTIMIZER.domain.values:
    hparams = {
        HP_NUM_FREEZED: num_freezed,
        HP_OPTIMIZER: optimizer,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-0
{'num_freezed': 3, 'optimizer': 'RMSprop'}


2022-03-30 11:22:26.431278: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-30 11:22:26.452570: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2801090000 Hz


41/41 [==============================] - 125s 3s/step - loss: 0.8357 - accuracy: 0.6638
--- Starting trial: run-1
{'num_freezed': 3, 'optimizer': 'adam'}
41/41 [==============================] - 116s 3s/step - loss: 0.8601 - accuracy: 0.6638
--- Starting trial: run-2
{'num_freezed': 5, 'optimizer': 'RMSprop'}
41/41 [==============================] - 96s 2s/step - loss: 1.0841 - accuracy: 0.2461
--- Starting trial: run-3
{'num_freezed': 5, 'optimizer': 'adam'}
41/41 [==============================] - 102s 2s/step - loss: 0.9620 - accuracy: 0.6638


## Launch tensorboard

In [12]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
